# 1. Install and Import Dependencies

In [104]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import requests
from bs4 import BeautifulSoup
import re

# 2. Instantiate Model

In [105]:
tokenizer = AutoTokenizer.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

model = AutoModelForSequenceClassification.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

# 3. Encode and Calculate Sentiment

In [106]:
tokens = tokenizer.encode('It was good but couldve been better. Great', return_tensors='pt')

In [107]:
result = model(tokens)

In [108]:
result.logits

tensor([[-2.7768, -1.2353,  1.4419,  1.9804,  0.4584]],
       grad_fn=<AddmmBackward0>)

In [109]:
int(torch.argmax(result.logits))+1

4

# 4. Collect Reviews

In [110]:
import requests
from bs4 import BeautifulSoup
import re

r = requests.get('https://quotes.toscrape.com/')
soup = BeautifulSoup(r.text, 'html.parser')

# Use regex to find classes that match 'text'
regex = re.compile('.*text.*')
results = soup.find_all('span', {'class': regex})

# Extract the text from the results
reviews = [result.text.strip() for result in results]

# Print the extracted quotes
for review in reviews:
    print(review)


“The world as we have created it is a process of our thinking. It cannot be changed without changing our thinking.”
“It is our choices, Harry, that show what we truly are, far more than our abilities.”
“There are only two ways to live your life. One is as though nothing is a miracle. The other is as though everything is a miracle.”
“The person, be it gentleman or lady, who has not pleasure in a good novel, must be intolerably stupid.”
“Imperfection is beauty, madness is genius and it's better to be absolutely ridiculous than absolutely boring.”
“Try not to become a man of success. Rather become a man of value.”
“It is better to be hated for what you are than to be loved for what you are not.”
“I have not failed. I've just found 10,000 ways that won't work.”
“A woman is like a tea bag; you never know how strong it is until it's in hot water.”
“A day without sunshine is like, you know, night.”


In [111]:
results

[<span class="text" itemprop="text">“The world as we have created it is a process of our thinking. It cannot be changed without changing our thinking.”</span>,
 <span class="text" itemprop="text">“It is our choices, Harry, that show what we truly are, far more than our abilities.”</span>,
 <span class="text" itemprop="text">“There are only two ways to live your life. One is as though nothing is a miracle. The other is as though everything is a miracle.”</span>,
 <span class="text" itemprop="text">“The person, be it gentleman or lady, who has not pleasure in a good novel, must be intolerably stupid.”</span>,
 <span class="text" itemprop="text">“Imperfection is beauty, madness is genius and it's better to be absolutely ridiculous than absolutely boring.”</span>,
 <span class="text" itemprop="text">“Try not to become a man of success. Rather become a man of value.”</span>,
 <span class="text" itemprop="text">“It is better to be hated for what you are than to be loved for what you are not.

In [112]:
r.text


'<!DOCTYPE html>\n<html lang="en">\n<head>\n\t<meta charset="UTF-8">\n\t<title>Quotes to Scrape</title>\n    <link rel="stylesheet" href="/static/bootstrap.min.css">\n    <link rel="stylesheet" href="/static/main.css">\n    \n    \n</head>\n<body>\n    <div class="container">\n        <div class="row header-box">\n            <div class="col-md-8">\n                <h1>\n                    <a href="/" style="text-decoration: none">Quotes to Scrape</a>\n                </h1>\n            </div>\n            <div class="col-md-4">\n                <p>\n                \n                    <a href="/login">Login</a>\n                \n                </p>\n            </div>\n        </div>\n    \n\n<div class="row">\n    <div class="col-md-8">\n\n    <div class="quote" itemscope itemtype="http://schema.org/CreativeWork">\n        <span class="text" itemprop="text">“The world as we have created it is a process of our thinking. It cannot be changed without changing our thinking.”</span>\

In [113]:
reviews

['“The world as we have created it is a process of our thinking. It cannot be changed without changing our thinking.”',
 '“It is our choices, Harry, that show what we truly are, far more than our abilities.”',
 '“There are only two ways to live your life. One is as though nothing is a miracle. The other is as though everything is a miracle.”',
 '“The person, be it gentleman or lady, who has not pleasure in a good novel, must be intolerably stupid.”',
 "“Imperfection is beauty, madness is genius and it's better to be absolutely ridiculous than absolutely boring.”",
 '“Try not to become a man of success. Rather become a man of value.”',
 '“It is better to be hated for what you are than to be loved for what you are not.”',
 "“I have not failed. I've just found 10,000 ways that won't work.”",
 "“A woman is like a tea bag; you never know how strong it is until it's in hot water.”",
 '“A day without sunshine is like, you know, night.”']

# 5. Load Reviews into DataFrame and Score

In [114]:
import numpy as np
import pandas as pd

In [115]:
df = pd.DataFrame(np.array(reviews), columns=['review'])

In [116]:
df['review'].iloc[0]

'“The world as we have created it is a process of our thinking. It cannot be changed without changing our thinking.”'

In [117]:
def sentiment_score(review):
    tokens = tokenizer.encode(review, return_tensors='pt')
    result = model(tokens)
    return int(torch.argmax(result.logits))+1

In [118]:
sentiment_score(df['review'].iloc[1])

5

In [119]:
df['sentiment'] = df['review'].apply(lambda x: sentiment_score(x[:512]))

In [120]:
df

,review,sentiment
0,“The world as we have created it is a process ...,5
1,"“It is our choices, Harry, that show what we t...",5
2,“There are only two ways to live your life. On...,5
3,"“The person, be it gentleman or lady, who has ...",1
4,"“Imperfection is beauty, madness is genius and...",1
5,“Try not to become a man of success. Rather be...,3
6,“It is better to be hated for what you are tha...,3
7,"“I have not failed. I've just found 10,000 way...",1
8,“A woman is like a tea bag; you never know how...,4
9,"“A day without sunshine is like, you know, nig...",5


In [121]:
df['sentiment']

0    5
1    5
2    5
3    1
4    1
5    3
6    3
7    1
8    4
9    5
Name: sentiment, dtype: int64

In [122]:
from emoji import emojize as em
# Define the classification function
def classify_sentiment(i):
    if i <= 2:
        return em(":pensive_face:")   # 😔 for negative
    elif i == 3:
        return em(":neutral_face:")   # 😐 for neutral
    else:
        return em(":smiling_face_with_smiling_eyes:")
    
df1 = df.copy()
# Replace the values in the 'sentiment' column
df1['sentiment'] = df1['sentiment'].apply(classify_sentiment)

# Show the updated DataFrame
print(df1)


                                              review sentiment
0  “The world as we have created it is a process ...         😊
1  “It is our choices, Harry, that show what we t...         😊
2  “There are only two ways to live your life. On...         😊
3  “The person, be it gentleman or lady, who has ...         😔
4  “Imperfection is beauty, madness is genius and...         😔
5  “Try not to become a man of success. Rather be...         😐
6  “It is better to be hated for what you are tha...         😐
7  “I have not failed. I've just found 10,000 way...         😔
8  “A woman is like a tea bag; you never know how...         😊
9  “A day without sunshine is like, you know, nig...         😊


In [123]:
# Define the classification function
def classify_sentient(i):
    if i <= 2:
        return 'negative'
    elif i == 3:
        return 'neutral'
    else:
        return 'positive'

# Replace the values in the 'sentiment' column
df['sentiment'] = df['sentiment'].apply(classify_sentient)

# Show the updated DataFrame
print(df)


                                              review sentiment
0  “The world as we have created it is a process ...  positive
1  “It is our choices, Harry, that show what we t...  positive
2  “There are only two ways to live your life. On...  positive
3  “The person, be it gentleman or lady, who has ...  negative
4  “Imperfection is beauty, madness is genius and...  negative
5  “Try not to become a man of success. Rather be...   neutral
6  “It is better to be hated for what you are tha...   neutral
7  “I have not failed. I've just found 10,000 way...  negative
8  “A woman is like a tea bag; you never know how...  positive
9  “A day without sunshine is like, you know, nig...  positive
